# 1. Raw Data

### 1.1 Create DB Conncection

In [ ]:
import db_connect
sql = db_connect.DatabaseConnect('CS581GROUP6','localhost','root','qwerty','3306')

# 2. Preprocessing

In [ ]:
from datetime import datetime
from datetime import timedelta  
import numpy as np
import pandas as pd
import math
import sys
from geopy.distance import geodesic

In [ ]:
max_delay_percent = 0.25
max_delay = 600
max_walking_time_percent = 0.10
max_walking_time = 240

pool_duration = 5

### 2.1 Mapping Coordinates to Destination ID

In [ ]:
def getdestid(lat,longt):
    return df_destinations.iloc[df_destinations.apply(lambda x: np.linalg.norm(np.array([lat, longt]) - np.array([x['destLat'],x['destLong']])), axis=1).idxmin(axis = 0)]['destID']

In [ ]:
def search_map_dest_id(lat_start, lat_end, long_start, long_end, dest_lat, dest_long, num_of_iter, num_of_iter_limit):
    
    if lat_start + (lat_end - lat_start)/2 >= dest_lat:
        lat_end = lat_start + (lat_end - lat_start)/2
    else:
        lat_start = lat_start + (lat_end - lat_start)/2

    if long_start + (long_end - long_start)/2 >= dest_long:
        long_end = long_start + (long_end - long_start)/2
    else:
        long_start = long_start + (long_end - long_start)/2
    num_of_iter = num_of_iter + 1
    if num_of_iter <= num_of_iter_limit:
        return search_map_dest_id(lat_start, lat_end, long_start, long_end, dest_lat, dest_long, num_of_iter,num_of_iter_limit)

    else:
        filtered_dest = df_destinations.loc[(df_destinations['destLat'] >= lat_start) & (df_destinations['destLat'] <= lat_end) & (df_destinations['destLong'] >= long_start) & (df_destinations['destLong'] <= long_end)]
        if len(filtered_dest) == 0:
            return False
        else:    
            return df_destinations.iloc[filtered_dest.apply(lambda x: np.linalg.norm(np.array([dest_lat,dest_long]) - np.array([x['destLat'],x['destLong']])), axis=1).idxmin(axis = 0)]['destID']

### 2.2 Pool Window Assignment

In [ ]:
def getPoolWindow(pickup_time, pool_start_time, pool_window_id):
    
    pool_end_time = pool_start_time + timedelta(minutes = pool_duration)
    
    if(pickup_time > pool_end_time):
        
        pool_window_id += 1
    
        while(pickup_time > pool_end_time):

            pool_start_time += timedelta(minutes = pool_duration)
            pool_end_time = pool_start_time + timedelta(minutes = pool_duration)
                
    return pool_start_time, pool_window_id

def poolWindowAssignment():
    
    pool_window_id = 1 
    pool_start_time = df['tpep_pickup_datetime'][0]
    pool_window_id_list = []
    req_pool_window_id_list = []
    pool_start_time_list = []
    pool_window_duration_list = []
    
    i = 0
    pool_window_id_list.append(pool_window_id)
    pool_start_time_list.append(pool_start_time)
    pool_window_duration_list.append(pool_duration)
    
    for pickup_datetime in df.tpep_pickup_datetime:
        
        pool_start_time, pool_window_id = getPoolWindow(pickup_datetime, pool_start_time, pool_window_id)
        req_pool_window_id_list.append(pool_window_id)

        #print('pool_window_id_list[',i,'] ',pool_window_id_list[i],' pool_window_id: ',pool_window_id)
        if(pool_window_id_list[i] != pool_window_id):
            pool_window_id_list.append(pool_window_id)
            pool_start_time_list.append(pool_start_time)
            pool_window_duration_list.append(pool_duration)
            i += 1
    
    df['pool_window_id'] = req_pool_window_id_list

    pool_window_data = {'poolingWindowID': pool_window_id_list, 'poolingStartTime': pool_start_time_list, 'poolingDuration': pool_window_duration_list}
    pool_window_df = pd.DataFrame(data=pool_window_data)
    return pool_window_df

### 2.3 Calculate Ride Time

In [ ]:
def getRideTime(time):
    time = sec_to_min(time.seconds)
    return time

### 2.4 Calculate Delay

In [ ]:
def getDelay(pickupTime, dropoffTime):
    tripTime = (dropoffTime - pickupTime).seconds
    trip_percent = tripTime * max_delay_percent
    return min(trip_percent, max_delay)

### 2.5 Calculate Walking Time

In [ ]:
def getWalkingTime(pickupTime, dropoffTime):
    tripTime = (dropoffTime - pickupTime).seconds
    trip_percent = tripTime * max_walking_time_percent
    return min(trip_percent, max_walking_time)

### 2.6 Calculate Cost for Individual Ride

In [ ]:
def sec_to_min(sec_time):
    return sec_time/60

In [ ]:
def meter_to_mile(meter_distance):
    return meter_distance/1609.34

In [ ]:
def cost_function(distance, time, isSharedTrip):
    #Convert milliseconds to minutes
    time = sec_to_min(time.seconds)
    #Convert meters to miles
    #distance = meter_to_mile(distance)
    if isSharedTrip:
        cost_per_mile = 1.00
        cost_per_min = 0.18
    else:
        cost_per_mile = 1.80
        cost_per_min = 0.28
    base_fare = 0
    booking_fee = 2.3
    min_fare = 7.3
    #print(distance, time)
    total_fare = (cost_per_mile * distance) + (cost_per_min * time) + base_fare + booking_fee + min_fare
    return total_fare

# Execute Preprocessing

In [ ]:
#df = pd.read_csv('RideSharingData/filtered_data2.csv')
df = pd.read_csv('RideSharingData/10_June_filtered_data.csv')
df_destinations = sql.query('select * from destination',True)

df['tpep_pickup_datetime'] = [datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S') 
                                          for pickup_time in df['tpep_pickup_datetime']]

df['tpep_dropoff_datetime'] = [datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S') 
                                          for dropoff_time in df['tpep_dropoff_datetime']]
import time
start_time = time.time()
#df['destination_id'] = df.iloc[:1000].apply(lambda x: getdestid(x['dropoff_latitude'],x['dropoff_longitude']),axis = 1)
df['destID'] = df.apply(lambda x: search_map_dest_id(40.69134374000000000000, 40.88140500000000000000, -74.04164664000000000000, -73.87790573000000000000, x['dropoff_latitude'], x['dropoff_longitude'], 0, 5),axis = 1)
elapsed_time = time.time() - start_time
print(elapsed_time)

df['indvRideTime'] = df.apply(lambda x: getRideTime(x['tpep_dropoff_datetime']-x['tpep_pickup_datetime']),axis = 1)
df['indvRideCost'] = df.apply(lambda x: cost_function(x['trip_distance'],x['tpep_dropoff_datetime']-x['tpep_pickup_datetime'], False),axis = 1)
df['maxDelay'] = df.apply(lambda x: getDelay(x['tpep_pickup_datetime'],x['tpep_dropoff_datetime']),axis = 1)
df['maxWalkTime'] = df.apply(lambda x: getWalkingTime(x['tpep_pickup_datetime'],x['tpep_dropoff_datetime']),axis = 1)

#cost_function(df['trip_distance'], df['tpep_dropoff_datetime']-df['tpep_pickup_datetime'], False)

In [ ]:
import time
start_time = time.time()
df.loc[df['destID']==False,'destID']=df.loc[df['destID']==False].apply(lambda x: getdestid(x['dropoff_latitude'], x['dropoff_longitude']),axis = 1)
elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
df.loc[df['destID']==False]

# 3. DF_Request Table

In [ ]:
pool_window_df = poolWindowAssignment()

In [ ]:
df['reqID'] =  df.index + 1
request_df = df[['reqID', 'numberOfPassengers', 'isWillingToWalk', 'destID', 'maxDelay','maxWalkTime','tpep_pickup_datetime','indvRideTime','trip_distance','indvRideCost','pool_window_id']].copy()

In [ ]:
request_df=request_df.rename(index=str, columns={'tpep_pickup_datetime':'requestTime','trip_distance':'indvRideDist','pool_window_id':'poolingWindowID'})

In [ ]:
request_df

In [ ]:
request_df.to_csv('RideSharingData/mapped_data.csv',sep=",");

In [ ]:
request_df.shape

# 4. Insert into SQL DB

In [ ]:
#pool_window_df
sql.insert(pool_window_df,'POOLING_WINDOW');

In [ ]:
sql.insert(request_df,'REQUESTS');

# Exceptions

In [ ]:
df_q = pd.merge(df_destinations, request_df, on='destID', how='inner')

In [ ]:
df_q.dtypes

In [ ]:
df.loc[df['total_amount']>500]

In [ ]:
df.loc[df['total_amount']<0]

In [ ]:
df.loc[df['indvRideCost']>400]

In [ ]:
df.loc[df['indvRideTime']>360].size

# Actual Individual Cost vs Calculated Individual Cost

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(len(df))


# Get current size
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
print ("Current size:", fig_size)
# Set figure width to 12 and height to 9
fig_size[0] = 16
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size


plt.plot(x, df['indvRideCost'])
plt.plot(x, df['total_amount'])

plt.legend(['calculated_cost','actual_cost'], loc='upper left')
plt.show()

# 4. Shareability Graph

### For each pool

In [ ]:
def getRequestListForEachPoolId():
    max_pool_window_id = df['pool_window_id'].max()
    all_req_list = []
    each_pool_req_list = []
    
    for x in range(1,  max_pool_window_id  ):
        each_pool_req_list = df.loc[df['pool_window_id'] == x]
        all_req_list.append(each_pool_req_list)
    return all_req_list

In [ ]:
def sharability_graph(pool_request_list):
    
    source_dest = (40.644190, -73.782366)
    taxi_capacity = 3
    graph = {}
    
    for index, req_A in pool_request_list.iterrows():
        
        dest_id_A = req_A['destID']
        if req_A['isWillingToWalk'] == 1:
            walk_time_A = req_A['maxWalkTime']
        else:
            walk_time_A = 0
        
        indv_ride_time_A = req_A['indvRideTime']
        max_delay_A = req_A['maxDelay']
        
        for idx, req_B in pool_request_list.iterrows():
            
            dest_id_B = req_B['destID']
            indv_ride_time_B = req_B['indvRideTime']
            max_delay_B = req_B['maxDelay']
            
            if req_B['isWillingToWalk'] == 1:
                walk_time_B = req_A['maxWalkTime']
            else:
                walk_time_B = 0
                
            if dest_id_A != dest_id_B and req_A['passenger_count'] + req_B['passenger_count'] <= taxi_capacity:
                
                poss_dest_A_list = []
                poss_dest_B_list = []
                
                if(apply_euclidean_elimination(req_A, req_B)==False):
                    continue
                
                poss_dest_A_list , poss_dest_B_list = possibleDestinations(dest_id_A, walk_time_A, dest_id_B, walk_time_B)

                max_benefit = 0
                   
                destIndexA, destIndexB = select_route(source_dest, poss_dest_A_list, poss_dest_B_list)
                print(dest_id_A, dest_id_B)
                print(destIndexA,destIndexB)
                print()
                #shared_time_A, shared_time_B = calculate_shared_time(source_dest, poss_dest_A, poss_dest_B)

#                 if shared_time_A <= indv_ride_time_A + max_delay_A and shared_time_B <= indv_ride_time_B + max_delay_B:
#                     benefit = compute_benefit(source_dest, poss_dest_A, poss_dest_B)
#                     if benefit > max_benefit:
#                         max_benefit = benefit
                                
#                 if req_A in graph:
#                     req_A_sharability_list = (graph['dest_id_A']).append([dest_id_B, max_benefit])
#                     graph['dest_id_A'] = req_A_sharability_list
#                 else:
#                     graph['dest_id_A'] = [[dest_id_B, max_benefit]]
                
    return graph

In [ ]:
def possibleDestinations(id1, walk1, id2, walk2):
    list1 = []
    list1.append(id1)
    list2 = []
    list2.append(id2)

    dest_tag = "DEST"
    id1 = int(id1.split(dest_tag)[1])
    id2 = int(id2.split(dest_tag)[1])

    if(walk1 <= 240):
        list1 = createDestinationList(dest_tag,id1)

    if(walk2 <= 240):
        list2 = createDestinationList(dest_tag,id2)

    return list1,list2

def createDestinationList(dest_tag, dest_id_num):
        list1 = []
        list1.append(dest_tag+str(dest_id_num))
        
        #Corners
        if(dest_id_num == 1):
            list1.append(dest_tag+"31")
            list1.append(dest_tag+"2")
        elif(dest_id_num == 30):
            list1.append(dest_tag+"29")
            list1.append(dest_tag+"60")
        elif(dest_id_num == 3331):
            list1.append(dest_tag+"3332")
            list1.append(dest_tag+"3301")
        elif(dest_id_num == 3360):
            list1.append(dest_tag+"3359")
            list1.append(dest_tag+"3330")

        #North edge
        elif(dest_id_num <= 30):
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(id+30))

        #south edge
        elif(dest_id_num >= 3330 and dest_id_num <= 3360):
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(dest_id_num-30))

        #west edge
        elif(dest_id_num%30 == 1):
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num-30))
            list1.append(dest_tag+str(dest_id_num+30))

        #east edge
        elif((dest_id_num%30) == 0):
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(dest_id_num+30))
            list1.append(dest_tag+str(dest_id_num-30))
        else:
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num+30))
            list1.append(dest_tag+str(dest_id_num-30))
        return list1

In [ ]:
def apply_euclidean_elimination(reqA, reqB):
    
    #Retrieve destID for request A & B
    destID_A = reqA['destID']
    destID_B = reqB['destID']
    
    #Get coordinates for destID_A and destID_B
    latA = df_destinations.loc[df_destinations['destID']==destID_A].iloc[0]['destLat']
    lonA = df_destinations.loc[df_destinations['destID']==destID_A].iloc[0]['destLong']
    latB = df_destinations.loc[df_destinations['destID']==destID_B].iloc[0]['destLat']
    lonB = df_destinations.loc[df_destinations['destID']==destID_B].iloc[0]['destLong']
    
    #Calculate euclidean distance in meters
    euc_distance = meter_to_mile(math.sqrt((latA - latB)**2 + (lonA - lonB)**2)*100*1000)

    #Using assumed average speed & euclidean distance calculate euclidean time from A to B
    avg_speed = 20
    calc_time_AtoB = euc_distance/(avg_speed/3600)
    
    #Check euclidean conditions for A to B and B to A
    time_StoA = reqA['indvRideTime']
    maxDelayA = reqA['maxDelay']
    time_StoB = reqB['indvRideTime']
    maxDelayB = reqA['maxDelay']
    if (time_StoA + calc_time_AtoB < time_StoB + maxDelayB) and (time_StoB + calc_time_AtoB < time_StoA + maxDelayA):
        return True
    else:
        return False

In [ ]:
def select_route(jfk, list1, list2):
    #check all pairs
    IDA = list1[0];
    IDB = list2[0];
    minDist = sys.maxsize
    
    #check all pairs
    for i in list1:
        for j in list2:

            aobject = df_destinations.loc[df_destinations['destID']==i]
            bobject = df_destinations.loc[df_destinations['destID']==j]

            a = (aobject["destLat"].values[0], aobject["destLong"].values[0])
            b = (bobject["destLat"].values[0], bobject["destLong"].values[0])
            
            jfkToA = geodesic(jfk, a).miles
            # jfkToB = geodesic(jfk, b).miles
            AToB = geodesic(a, b).miles

            totalAB = jfkToA+AToB
            # totalBA = jfkToB+AToB

            #update indices if smaller route found, update minDist 
            if(totalAB < minDist):
                minDist = totalAB
                IDA = i
                IDB = j

    #return indices in order
    return IDA, IDB

In [ ]:
all_req_list = getRequestListForEachPoolId()
graph =[]
for pool_request_list in all_req_list:
    graph = sharability_graph(pool_request_list)

In [ ]:
#len(all_req_list)
a = 0
for pool in all_req_list:
    a+=len(pool)
print(a)

In [ ]:
import math
latA = df_destinations.loc[df_destinations['destID']=='DEST2000'].iloc[0]['destLat']
lonA = df_destinations.loc[df_destinations['destID']=='DEST2000'].iloc[0]['destLong']
latB = df_destinations.loc[df_destinations['destID']=='DEST270'].iloc[0]['destLat']
lonB = df_destinations.loc[df_destinations['destID']=='DEST270'].iloc[0]['destLong']
#type(latA)
print(str(latA)+"\t"+str(lonA))
print(str(latB)+"\t"+str(lonB))
euc_distance = meter_to_mile(math.sqrt((latA - latB)**2 + (lonA - lonB)**2)*100*1000)
print(euc_distance)
avg_speed = 20
calc_time = euc_distance/(avg_speed/3600)
print(calc_time)
#hav_distance = meter_to_mile(haversine(lonA, latA, lonB, latB)*1000)

In [ ]:
print(df_destinations.loc[df_destinations['destID']=='DEST300'])

In [ ]:
a = (df_destinations.iloc[300]["destLat"], df_destinations.iloc[300]["destLong"])
b = (df_destinations.iloc[400]["destLat"], df_destinations.iloc[400]["destLong"])

In [ ]:
print(a)
print(b)

In [ ]:
aObject = df_destinations.loc[df_destinations['destID']=='DEST300']
print(list(aObject['destLat'], aObject['destLong']))